In [52]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Linear Regression

## Load in Data

In [77]:
diabetes = load_diabetes()
diabetesX = diabetes.data
diabetesY = diabetes.target

## Math notes
**L(b) = 1/2n ||Xb - y||^2**

b <- vector with beta coefficients 

X <- Matrix with n rows and d+1 columns, each row of X is augmented feature vector (1 at beginning of vector)

y <- A vector with all target values


**L'(b) = 1/n X^T (Xb - y) = 0** =

X^T Xb = X^T y    <- solve for beta and then `BAM` we're there baby

# Deriving regression as done above

In [92]:
# Each row of diabetesX is an augmented feature vector 
#   Side note though this does remove the sex column if you run a pandas .corr() you see there is almost 
#   no correlation with sex and our target value. So in case we do not mind.
diabetesX[:,0] = np.ones(diabetesX.shape[0])
# From this we then compute the coefficient matrix in the normal equations (which is now 10 x 10)
diabetesXTX = diabetesX.T @ diabetesX
# Right hand side of the normal equations
diabetesXTY = diabetesX.T @ diabetesY
# Solve the normal equations
diabetesBeta = np.linalg.solve(diabetesXTX, diabetesXTY)

## Putting it all together
We now put together all of the components into a pandas dataframe and use pandas functionality and our previously calcualted beta values to find the predicted values and can see them in comparison to the real Y values. Having it in a pandas df also would allow us to easily calcuate the MSE and other statistical calcuations around accuracy if we so desired.

In [112]:
linearRegressionTest = pd.DataFrame(diabetesX)
linearRegressionTest['realYValues'] = diabetesY
linearRegressionTest['predictedYValues'] = diabetesBeta[0] + diabetesBeta[1]*linearRegressionTest[1] + diabetesBeta[2]*linearRegressionTest[2]+ diabetesBeta[3]*linearRegressionTest[3]+ diabetesBeta[4]*linearRegressionTest[4]+ diabetesBeta[5]*linearRegressionTest[5]+ diabetesBeta[6]*linearRegressionTest[6]+ diabetesBeta[7]*linearRegressionTest[7]+ diabetesBeta[8]*linearRegressionTest[8]+ diabetesBeta[9]*linearRegressionTest[9]
linearRegressionTest

,0,1,2,3,4,5,6,7,8,9,realYValues,predictedYValues
0,1.0,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0,206.477690
1,1.0,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0,68.325985
2,1.0,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0,177.798162
3,1.0,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0,166.138490
4,1.0,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0,128.592054
...,...,...,...,...,...,...,...,...,...,...,...,...
437,1.0,0.050680,0.019662,0.059744,-0.005697,-0.002566,-0.028674,-0.002592,0.031193,0.007207,178.0,193.230846
438,1.0,0.050680,-0.015906,-0.067642,0.049341,0.079165,-0.028674,0.034309,-0.018118,0.044485,104.0,100.980006
439,1.0,0.050680,-0.015906,0.017282,-0.037344,-0.013840,-0.024993,-0.011080,-0.046879,0.015491,132.0,121.628192
440,1.0,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044528,-0.025930,220.0,211.441064


# Logistic regression

## Read in and look over data

Here we clean and vectorize what data we can, we also remove which ever columns we believe wont be helpful after we view the correletion matrix given by titanicTrainData.corr(). From this we are left with Pclass, Sex, SibSp, and Fare.

In [149]:
titanicTestData = pd.read_csv('data/titanic_test.csv')
titanicTestData = titanicTestData[['Pclass', 'Sex', 'SibSp', 'Fare', 'Age']]
titanicTestData['Sex'] = pd.get_dummies(titanicTestData['Sex'],drop_first=True)

titanicTrainData = pd.read_csv('data/titanic_train.csv')
titanicTrainData = titanicTrainData[['Survived', 'Pclass', 'Sex', 'SibSp', 'Fare', 'Age']]
titanicTrainData['Sex'] = pd.get_dummies(titanicTrainData['Sex'],drop_first=True)